### Load required packages

In [1]:
import numpy as np
import pandas as pd
import zipfile, pickle
from sklearn.metrics import precision_recall_curve, auc
from convert_pIC50_to_activity import bin_predicted_pIC50, bin_observed_pIC50

In [7]:
from math import ceil
def enrichment_factor(y_true, y_score, percentage=1, pos_label=None, kind='fold'):
    """Computes enrichment factor for given percentage, i.e. EF_1% is
    enrichment factor for first percent of given samples. This function assumes
    that results are already sorted and samples with best predictions are first.

    Parameters
    ----------
    y_true : array, shape=[n_samples]
        True binary labels, in range {0,1} or {-1,1}. If positive label is
        different than 1, it must be explicitly defined.

    y_score : array, shape=[n_samples]
        Scores for tested series of samples

    percentage : int or float
        The percentage for which EF is being calculated

    pos_label: int
        Positive label of samples (if other than 1)

    kind: 'fold' or 'percentage' (default='fold')
        Two kinds of enrichment factor: fold and percentage.
        Fold shows the increase over random distribution (1 is random, the
        higher EF the better enrichment). Percentage returns the fraction of
        positive labels within the top x% of dataset.

    Returns
    -------
    ef : float
        Enrichment Factor for given percenage in range 0:1
    """
    if pos_label is None:
        pos_label = 1
    labels = y_true == pos_label
    assert labels.sum() > 0, "There are no correct predicions. Double-check the pos_label"
    assert len(labels) > 0, "Sample size must be greater than 0"
    # calculate fraction of positve labels
    n_perc = int(ceil(percentage / 100. * len(labels)))
    out = labels[:n_perc].sum() / n_perc
    if kind == 'fold':
        out /= (labels.sum() / len(labels))
    return out

In [1]:
def bin_predicted_pIC50(row):
    if row['Predicted_pIC50'] >= 5:
        val = 'Active'
    else:
        val = "Inactive"
    return val

def bin_observed_pIC50(row):
    if row['Observed_pIC50'] >= 5:
        val = 'Active'
    else:
        val = 'Inactive'
    return val

### SVM GRID SF Model (Training actives + RandomDecoys)

In [2]:
filename = "../models/GRID_SVM_SFs_training_actives_and_RandomDecoys.sav"
GRID_SVM_SF = pickle.load(open(filename, 'rb'))

/home/junaid/miniconda3/envs/Jupyter_Dock/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator SVR from version 0.24.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


### Load GRID Features

In [3]:
test_rdkit_grid_features_pubchems = np.load("../data/test_set/RDKit_Grid_Feature_Test_PubChem.zip")
test_rdkit_grid_features_pubchems = pd.DataFrame(test_rdkit_grid_features_pubchems['RDKit_Grid_Feature_Test_PubChem'])

PubChems_Labels = pd.read_csv("../data/test_set/PubChem_IDs.csv")

### Predict on Test Set

In [4]:
prediction_test_rdkit_grid_svm = GRID_SVM_SF.predict(test_rdkit_grid_features_pubchems)

### Get AUC-PR

In [5]:
svm_rdkit_grid_result = pd.DataFrame({"Predicted_pIC50": prediction_test_rdkit_grid_svm,
                                     "Observed_pIC50": list(PubChems_Labels['pIC50'])})
svm_rdkit_grid_result['Predicted_Activity'] = svm_rdkit_grid_result.apply(bin_predicted_pIC50, axis = 1)
svm_rdkit_grid_result['Observed_Activity'] = svm_rdkit_grid_result.apply(bin_observed_pIC50,axis = 1)
from sklearn.metrics import precision_recall_curve, auc
svm_rdkit_grid_result['normalized_scores'] = (svm_rdkit_grid_result['Predicted_pIC50'] - svm_rdkit_grid_result['Predicted_pIC50'].min()) / (svm_rdkit_grid_result['Predicted_pIC50'].max() - svm_rdkit_grid_result['Predicted_pIC50'].min())
precision_rdkit_grid_svm, recall_rdkit_grid_svm, threshold_rdkit_grid_svm = precision_recall_curve(svm_rdkit_grid_result['Observed_Activity'], svm_rdkit_grid_result['normalized_scores'], pos_label = 'Active')
svm_rdkit_grid_precision_recall = pd.DataFrame({"Precision": precision_rdkit_grid_svm,
                                               "Recall": recall_rdkit_grid_svm})
print(auc(recall_rdkit_grid_svm, precision_rdkit_grid_svm))

0.9419846969849105


### Get EF1% 

In [8]:
data_active_score = pd.DataFrame(svm_rdkit_grid_result['Predicted_pIC50'])
data_active_score['activity'] = svm_rdkit_grid_result['Observed_Activity']
data_active_score.sort_values('Predicted_pIC50', inplace=True, ascending=False)
enrichment_value= round(enrichment_factor(data_active_score['activity'], data_active_score['Predicted_pIC50'], percentage=1, pos_label='Active'))
enrichment_value

63